In [1]:
import numpy as np
import pandas as pd
import gurobipy as gb
from gurobipy import GRB

In [2]:
# We don't *need* these to be arrays, but its closer to what we'll have in the final programme.
c = np.array([
    [0,6,4,2,4],
    [6,0,5,5,2],
    [4,5,0,4,6],
    [2,5,4,0,3],
    [4,2,6,3,0]
]) # This is a symmetric matrix as expected, since c_ij == c_ji.
q = np.array([0,3,5,4,6])
Q = 9
v = 5
V = set(range(v))
def Del(p):
    return V - set([p])
N = Del(0)

In [3]:
from itertools import product
A = product(N,N)
A = filter(lambda tup: tup[0] != tup[1], A)

In [4]:
m = gb.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-12


In [5]:
x = m.addVars(v,v,vtype=GRB.BINARY, name='x')

In [6]:
#Constraints are coded in the order they appear in the final report

In [7]:
m.setObjective(sum([10 * c[i,j] * x[i,j] for i,j in product(V,V)]), GRB.MINIMIZE)

In [8]:
#Source and sink constraints
for i in N:
    m.addConstr(sum([x[i,j] for j in Del(i)]) == 1)
for j in N:
    m.addConstr(sum([x[i,j] for i in Del(j)]) == 1 )

In [9]:
#Construction of L loops
L = m.addVar(lb=0.0, vtype=GRB.INTEGER, name='L')
m.addConstr(sum([x[0,j] for j in Del(0)]) == L)

<gurobi.Constr *Awaiting Model Update*>

In [10]:
#Capacity constraints
u = m.addVars(V, vtype=GRB.INTEGER, name='u')
for i,j in A:
    m.addConstr(u[i] - u[j] + Q*x[i,j] <= Q - q[j])
for i in V:
    m.addConstr(u[i] >= q[i])

In [11]:
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 26 rows, 31 columns and 78 nonzeros
Model fingerprint: 0xc5327254
Variable types: 0 continuous, 31 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [2e+01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective 320.0000000
Presolve removed 6 rows and 15 columns
Presolve time: 0.00s
Presolved: 20 rows, 16 columns, 60 nonzeros
Variable types: 0 continuous, 16 integer (12 binary)
Found heuristic solution: objective 220.0000000

Root relaxation: objective 1.700000e+02, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  170.00000    0    6  220.00000  170.00000  22.7%    

In [12]:
for var in m.getVars():
    print(var.varName, var.x)
    
print('Total cost: {}'.format(m.ObjVal))

x[0,0] 0.0
x[0,1] 1.0
x[0,2] 0.0
x[0,3] 1.0
x[0,4] 0.0
x[1,0] 0.0
x[1,1] 0.0
x[1,2] 0.0
x[1,3] 0.0
x[1,4] 1.0
x[2,0] 1.0
x[2,1] 0.0
x[2,2] 0.0
x[2,3] 0.0
x[2,4] 0.0
x[3,0] 0.0
x[3,1] 0.0
x[3,2] 1.0
x[3,3] 0.0
x[3,4] 0.0
x[4,0] 1.0
x[4,1] 0.0
x[4,2] 0.0
x[4,3] 0.0
x[4,4] 0.0
L 2.0
u[0] 0.0
u[1] 3.0
u[2] 9.0
u[3] 4.0
u[4] 9.0
Total cost: 220.0


Shreyas' Notes:
1. We have the data we need, and I have been able to clean the data.
2. The small example above is in line with the expected optimal solution to this problem.
3. I tried using the programme with the entirety of the data from the Food Bank, but it took an unreasonable amounnt of time before I aborted it. I shall try again after performing a clustering of the data.